In [2]:
# Sonu Giri
# sonugiri1043@gmail.com
#
# https://keras.io/

!pip install -q keras
import keras

import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Input, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
from keras.layers.advanced_activations import LeakyReLU
from keras.layers import Activation, Conv2D, Input, MaxPooling2D, BatchNormalization, Flatten, Lambda
from keras.layers.merge import concatenate

# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k
# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

# Hyperparameters
batch_size = 256
num_classes = 10
epochs = 50
l = 10

Using TensorFlow backend.


In [3]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 339s 2us/step


In [32]:
print( img_height, img_width, channel )

32 32 3


In [0]:
def space_to_depth_x2(x):
    return tf.space_to_depth(x, block_size=8)

In [0]:
input = Input(shape=(img_height, img_width, channel,))
 
def blockBuilder( blockNum, inputToBlock, layerInEachBlock=5 ):
  # layer 1
  name = str( blockNum* layerInEachBlock + 1 )
  layer1 = Conv2D(32, (3,3), strides=(1,1), padding='same', name='conv_%s' % name, use_bias=False)(inputToBlock)
  layer1 = BatchNormalization(name='norm_%s' % name )(layer1)
  layer1 = LeakyReLU(name="leaky_relu_%s" % name, alpha=0.1)(layer1)
  
  # layer 2
  name = str( blockNum* layerInEachBlock + 2 )
  layer2 = Conv2D(64, (3,3), strides=(1,1), padding='same', name='conv_%s' % name, use_bias=False)(layer1)
  layer2 = BatchNormalization(name='norm_%s' % name )(layer2)
  layer2 = LeakyReLU(name="leaky_relu_%s" % name, alpha=0.1)(layer2)
  
  # layer 3
  name = str( blockNum* layerInEachBlock + 3 )
  layer3 = Conv2D(128, (3,3), strides=(1,1), padding='same', name='conv_%s' % name, use_bias=False)(layer2)
  layer3 = BatchNormalization(name='norm_%s' % name )(layer3)
  layer3 = LeakyReLU(name="leaky_relu_%s" % name, alpha=0.1)(layer3)
  
  # layer 4
  name = str( blockNum* layerInEachBlock + 4 )
  layer4 = Conv2D(256, (3,3), strides=(1,1), padding='same', name='conv_%s' % name, use_bias=False)(layer3)
  layer4 = BatchNormalization(name='norm_%s' % name )(layer4)
  layer4 = LeakyReLU(name="leaky_relu_%s" % name, alpha=0.1)(layer4)
  
  # layer 5
  name = str( blockNum* layerInEachBlock + 5 )
  layer5 = Conv2D(512, (3,3), strides=(1,1), padding='same', name='conv_%s' % name, use_bias=False)(layer4)
  layer5 = BatchNormalization(name='norm_%s' % name )(layer5)
  layer5 = LeakyReLU(name="leaky_relu_%s" % name, alpha=0.1)(layer5)
 
  # MP after layer 5
  layer5 = MaxPooling2D(name="max_pool_%s" % name, pool_size=(2, 2))(layer5)
  return layer5

# Block num starts from 0
# block 1
block0 = blockBuilder( blockNum=0, inputToBlock=input )
skip_connection = block0
# block 2
block1 = blockBuilder( blockNum=1, inputToBlock=block0 )
# block 3
block2 = blockBuilder( blockNum=2, inputToBlock=block1 )
# block 4
block3 = blockBuilder( blockNum=3, inputToBlock=block2 )

# skip connection from block 1
skip_connection = Conv2D(64, (1,1), strides=(1,1), padding='same', name='conv_skip', use_bias=False)(skip_connection)
skip_connection = BatchNormalization(name='norm_skip')(skip_connection)
skip_connection = LeakyReLU(name="relu_skip", alpha=0.1)(skip_connection)
skip_connection = Lambda( space_to_depth_x2, name="lambda_skip")(skip_connection)

concatLayer = concatenate([skip_connection, block3])

# flatten layer
flattenLayer = Flatten()(concatLayer)
output = Dense(num_classes, activation='softmax')(flattenLayer)

In [39]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_10 (InputLayer)           (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 32, 32, 32)   864         input_10[0][0]                   
__________________________________________________________________________________________________
norm_1 (BatchNormalization)     (None, 32, 32, 32)   128         conv_1[0][0]                     
__________________________________________________________________________________________________
leaky_relu_1 (LeakyReLU)        (None, 32, 32, 32)   0           norm_1[0][0]                     
__________________________________________________________________________________________________
conv_2 (Co

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [41]:
# epoch 1 - 10
batch_size=128
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 233s 5ms/step - loss: 1.6464 - acc: 0.4834 - val_loss: 2.3342 - val_acc: 0.5128
Epoch 2/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.9824 - acc: 0.6606 - val_loss: 0.9638 - val_acc: 0.6668
Epoch 3/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.7975 - acc: 0.7227 - val_loss: 1.1045 - val_acc: 0.6487
Epoch 4/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.6889 - acc: 0.7597 - val_loss: 1.3738 - val_acc: 0.6455
Epoch 5/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.5963 - acc: 0.7955 - val_loss: 1.0231 - val_acc: 0.6683
Epoch 6/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.5096 - acc: 0.8239 - val_loss: 0.8145 - val_acc: 0.7410
Epoch 7/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.4253 - acc: 0.8529 - val_loss: 0.9485 - 

In [42]:
# epoch 11 - 20
batch_size=128
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.1568 - acc: 0.9444 - val_loss: 1.1658 - val_acc: 0.7285
Epoch 2/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.1286 - acc: 0.9545 - val_loss: 1.1984 - val_acc: 0.7235
Epoch 3/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0956 - acc: 0.9671 - val_loss: 1.3386 - val_acc: 0.7273
Epoch 4/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0886 - acc: 0.9693 - val_loss: 1.4234 - val_acc: 0.7251
Epoch 5/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0865 - acc: 0.9688 - val_loss: 1.9879 - val_acc: 0.6597
Epoch 6/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0761 - acc: 0.9732 - val_loss: 1.3055 - val_acc: 0.7480
Epoch 7/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0566 - acc: 0.9808 - val_loss: 1.3181 - 

In [43]:
# epoch 21 - 30
batch_size=128
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0328 - acc: 0.9888 - val_loss: 2.0006 - val_acc: 0.6982
Epoch 2/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0411 - acc: 0.9860 - val_loss: 1.6389 - val_acc: 0.7360
Epoch 3/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0599 - acc: 0.9788 - val_loss: 1.5536 - val_acc: 0.7337
Epoch 4/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0617 - acc: 0.9788 - val_loss: 1.5100 - val_acc: 0.7411
Epoch 5/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0378 - acc: 0.9871 - val_loss: 1.6321 - val_acc: 0.7387
Epoch 6/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0345 - acc: 0.9880 - val_loss: 1.6626 - val_acc: 0.7320
Epoch 7/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0260 - acc: 0.9910 - val_loss: 1.7616 - 

In [44]:
# epoch 31 - 40
batch_size=128
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 225s 4ms/step - loss: 0.0268 - acc: 0.9903 - val_loss: 1.6831 - val_acc: 0.7479
Epoch 2/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0389 - acc: 0.9867 - val_loss: 1.6753 - val_acc: 0.7378
Epoch 3/10
50000/50000 [==============================] - 225s 4ms/step - loss: 0.0348 - acc: 0.9879 - val_loss: 1.6464 - val_acc: 0.7430
Epoch 4/10
50000/50000 [==============================] - 225s 4ms/step - loss: 0.0215 - acc: 0.9926 - val_loss: 1.7815 - val_acc: 0.7414
Epoch 5/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0226 - acc: 0.9922 - val_loss: 1.8032 - val_acc: 0.7330
Epoch 6/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0307 - acc: 0.9888 - val_loss: 1.9689 - val_acc: 0.7265
Epoch 7/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0306 - acc: 0.9893 - val_loss: 1.8159 - 

In [45]:
# epoch 41 - 50
batch_size=128
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0187 - acc: 0.9934 - val_loss: 1.7842 - val_acc: 0.7444
Epoch 2/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0212 - acc: 0.9922 - val_loss: 1.8046 - val_acc: 0.7477
Epoch 3/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0268 - acc: 0.9909 - val_loss: 2.0151 - val_acc: 0.7242
Epoch 4/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0281 - acc: 0.9900 - val_loss: 1.9072 - val_acc: 0.7368
Epoch 5/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0140 - acc: 0.9951 - val_loss: 1.9106 - val_acc: 0.7412
Epoch 6/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0234 - acc: 0.9920 - val_loss: 1.8773 - val_acc: 0.7393
Epoch 7/10
50000/50000 [==============================] - 224s 4ms/step - loss: 0.0235 - acc: 0.9918 - val_loss: 1.9124 - 

In [46]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 18s 2ms/step
Test loss: 1.9549589955329896
Test accuracy: 0.7438


In [47]:
# Save the trained weights in to .h5 format
model.save_weights("Yolo_Basic_model_.h5")
print("Saved the model to disk")

Saved the model to disk
